In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from retrying import retry

# 変更されたURL（福生市）
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?url=%2Fchintai%2Fichiran%2FFR301FC001%2F&ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&cb=0.0&ct=9999999&et=9999999&mb=0&mt=9999999&cn=9999999&ta=13&sc=13218&page={}"

@retry(wait_fixed=1000, stop_max_attempt_number=3)
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

all_data = []
page = 1

while True:
    # URLを定義
    url = base_url.format(page)

    # HTMLを取得
    soup = get_html(url)

    # 全ての物件を抽出
    items = soup.findAll("div", {"class": "cassetteitem"})
    print("page", page, "items", len(items))

    if len(items) == 0:
        break

    # 各物件を処理
    for item in items:
        stations = item.findAll("div", {"class": "cassetteitem_detail-text"})

        # 各駅を処理
        for station in stations:
            # 変数を定義
            base_data = {}

            # 基本情報を収集
            base_data["名称"] = item.find("div", {"class": "cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリー"] = item.find("div", {"class": "cassetteitem_content-label"}).getText().strip()
            base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).getText().strip()
            base_data["アクセス"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()

            # 各部屋を処理
            tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

            for tbody in tbodys:
                data = base_data.copy()

                data["階数"] = tbody.findAll("td")[2].getText().strip()

                data["家賃"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()

                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()

                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()

                data["URL"] = "https://suumo.jp" + tbody.findAll("td")[8].find("a").get("href")

                all_data.append(data)

    # 次のページへ
    next_button = soup.find("a", string="次へ")
    if next_button is None:
        break

    page += 1

# DataFrameに変換
df = pd.DataFrame(all_data)


page 1 items 50
page 2 items 50
page 3 items 50
page 4 items 50
page 5 items 50
page 6 items 50
page 7 items 50
page 8 items 50
page 9 items 44


In [4]:
df

,名称,カテゴリー,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,URL
0,フォンターナ・ルーチェ,賃貸アパート,東京都福生市武蔵野台１,ＪＲ青梅線/福生駅 歩11分,築8年,2階建,1階,5.75万円,2800円,-,-,1K,27.29m2,https://suumo.jp/chintai/jnc_000082616750/?bc=...
1,フォンターナ・ルーチェ,賃貸アパート,東京都福生市武蔵野台１,ＪＲ青梅線/福生駅 歩11分,築8年,2階建,1階,5.75万円,2800円,-,-,1K,27.29m2,https://suumo.jp/chintai/jnc_000082901979/?bc=...
2,フォンターナ・ルーチェ,賃貸アパート,東京都福生市武蔵野台１,ＪＲ八高線/東福生駅 歩5分,築8年,2階建,1階,5.75万円,2800円,-,-,1K,27.29m2,https://suumo.jp/chintai/jnc_000082616750/?bc=...
3,フォンターナ・ルーチェ,賃貸アパート,東京都福生市武蔵野台１,ＪＲ八高線/東福生駅 歩5分,築8年,2階建,1階,5.75万円,2800円,-,-,1K,27.29m2,https://suumo.jp/chintai/jnc_000082901979/?bc=...
4,フォンターナ・ルーチェ,賃貸アパート,東京都福生市武蔵野台１,,築8年,2階建,1階,5.75万円,2800円,-,-,1K,27.29m2,https://suumo.jp/chintai/jnc_000082616750/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,斉藤店舗,賃貸その他,東京都福生市大字熊川,ＪＲ青梅線/牛浜駅 歩14分,築34年,2階建,1階,13.75万円,-,41.25万円,-,ワンルーム,65.1m2,https://suumo.jp/chintai/jnc_000061204109/?bc=...
2009,斉藤店舗,賃貸その他,東京都福生市大字熊川,ＪＲ青梅線/拝島駅 歩15分,築34年,2階建,1階,13.75万円,-,41.25万円,-,ワンルーム,65.1m2,https://suumo.jp/chintai/jnc_000061204109/?bc=...
2010,ＪＲ青梅線 拝島駅 2階建 築37年,賃貸一戸建て,東京都福生市南田園１,ＪＲ青梅線/拝島駅 歩17分,築37年,2階建,1-2階,25万円,-,25万円,25万円,5LDK,166.03m2,https://suumo.jp/chintai/jnc_000080645129/?bc=...
2011,ＪＲ青梅線 拝島駅 2階建 築37年,賃貸一戸建て,東京都福生市南田園１,ＪＲ五日市線/熊川駅 歩13分,築37年,2階建,1-2階,25万円,-,25万円,25万円,5LDK,166.03m2,https://suumo.jp/chintai/jnc_000080645129/?bc=...


In [5]:
df.to_csv('fussa2.csv', index=False)